# S-parameter Correction and Manipulation
Examples of how to apply a 12-term and 8-term correction model to s-parameter data. In addition the transformation between S and T matrices. 

In [6]:
from pyMeasure import *
import os
import numpy as np

In [4]:
# import an s-parameter file to play with
os.chdir(TESTS_DIRECTORY)
two_port=S2PV1("20160301_30ft_cable_0.s2p")
two_port.show()

In [22]:
# Now create a numpy matrix of sparameters to do T-matrix conversion, should be added to S2PV1 model
matrix_form=[[row[0],np.matrix(row[1:]).reshape((2,2))] for row in two_port.sparameter_complex]

In [53]:
# Convert to T_matrix form, also should be added to S2PV1
t_matrix=[]
for row in matrix_form:
    frequency=row[0]
    m=row[1]
    T11=-np.linalg.det(m)/m[1,0]
    T12=m[0,0]/m[1,0]
    T21=-m[1,1]/m[1,0]
    T22=1/m[1,0]
    t_matrix.append([frequency,np.matrix([[T11,T21],[T21,T22]])])

In [57]:
def S_to_T(S_list):
    """Converts S-parameters into a T Matrix. Input form should be in frequency, np.matrix([[S11,S12],[S21,S22]])
    format. Returns a list in [frequency, np.matrix] format """
    t_matrix=[]
    for row in S_list:
        frequency=row[0]
        m=row[1]
        T11=-np.linalg.det(m)/m[1,0]
        T12=m[0,0]/m[1,0]
        T21=-m[1,1]/m[1,0]
        T22=1/m[1,0]
        t_matrix.append([frequency,np.matrix([[T11,T12],[T21,T22]])])
    return t_matrix

def T_to_S(T_list):
    """Converts T Matrix into S parameters. Input form should be in frequency, np.matrix([[T11,T12],[T21,T22]])
    format. Returns a list in [frequency, np.matrix] format."""
    S_list=[]
    for row in T_list:
        frequency=row[0]
        m=row[1]
        S11=m[0,1]/m[1,1]
        S12=np.linalg.det(m)/m[1,1]
        S21=1/m[1,1]
        S22=-m[1,0]/m[1,1]
        S_list.append([frequency,np.matrix([[S11,S12],[S21,S22]])])
    return S_list

In [62]:
# now we try it on the matrix form
t_matrix=S_to_T(matrix_form)
s_again=T_to_S(t_matrix)

# lets compare the results
print("Before conversion to T the first row is {0}".format(matrix_form[0]))
print("After the conversion to T the first row is {0}".format(t_matrix[0]))
print("Converting back to S the first row is {0}".format(s_again[0]))
print("The assertion S before is equal to S after is {0} ".format(matrix_form[0][1].all()==s_again[0][1].all()))

Before conversion to T the first row is [10.0, matrix([[ 0.00850644+0.00190443j, -0.00109549+0.10089327j],
        [-0.00065892+0.10100842j,  0.00618306+0.00530229j]])]
After the conversion to T the first row is [10.0, matrix([[-0.00165583+0.10131766j,  0.01830402-0.08433454j],
        [-0.05209200+0.06155311j, -0.06457989-9.89974361j]])]
Converting back to S the first row is [10.0, matrix([[ 0.00850644+0.00190443j, -0.00109549+0.10089327j],
        [-0.00065892+0.10100842j,  0.00618306+0.00530229j]])]
The assertion S before is equal to S after is True 


In [63]:
m[1,1]

(-0.071433839286603504+0.039971326812197797j)

<img src="./Calibration_Example_Files/Twelve_Term_Correction.png" />
VNA Calibration Coefficients

StatistiCAL stores VNA calibration coefficients in the following order:

frequency  Edf  Esf  Erf  Exf  Elf  Etf  Edr  Esr  Err  Exr  Elr  Etr.

The value of the frequency is in gigahertz. The other values are expressed as real/imaginary pairs.




In [ ]:
# Now we need to have a model for the error coefficients and a function that applies them to the s-parameters
class TwelveTermErrorModel(AsciiDataTable):
    """TwevleTermErrorModel holds the error coefficients for a twelve term model. The VNA calibration coefficeients 
    are presumed to be stored in the following order frequency Edf Esf Erf Exf Elf Etf Edr Esr Err Exr Elr Etr, where
    all coefficients are in Real-Imaginary format. """
    pass

    